In [146]:
import matplotlib.pyplot as plt
import numpy as  np
from scipy.interpolate import CubicSpline 
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm
from scipy.stats import norm, gamma
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 

In [147]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
import warnings
from time import time
import plotly.graph_objects as go
from itertools import product
from tqdm import tqdm
from scipy.interpolate import CubicSpline 

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 

warnings.simplefilter('ignore')

In [149]:
# We set time to maturity to 1 year
T = 1

# We create the four functions that will help us create our sample
def at_the_money_vol(z1,v_ref=0.2,v_width=0.75):
    v = v_ref * np.exp(v_width*z1 - 1/2*v_width**2)
    return v
    
def skew(z2):
    beta = norm.cdf(z2)
    return beta

def vol_of_vol(z3, V_mult = 4):
    return np.sqrt(2*(V_mult-1)*norm.cdf(z3)/T)

def strike(z4, v, gamma_vol, K_width=0.2):
    g = (K_width**2 * v**2 * T * (1 + gamma_vol**2 * T/2))**(-1)
    K = gamma.ppf(norm.cdf(z4), g) / g 
    return K


In [150]:
def check_region(v, beta, gamma, K):
    # A compléter
    return 0

In [151]:
#We fix the seed
np.random.seed(123)

#We create the sample
def generator_sample(n_samples=10):
    Df  = pd.DataFrame(columns = ['v', 'beta', 'gamma_vol', 'K'],index =[i for i in  range(n_samples)])
    for i in range(n_samples):
        # Draw standard normal variables
        z1, z2, z3, z4 = np.random.normal(size=4)

        # Compute the four variables
        v = at_the_money_vol(z1)
        beta = skew(z2)
        gamma_vol = vol_of_vol(z3)
        K = strike(z4, v, gamma_vol)
        Df.loc[i, :] = [v, beta, gamma_vol, K]
        
        # Check the region of the sample
        #region = check_region(z1,z2,z3,z4)

        #if check_region == 0:
        #    yield None
        #else:
    return Df
    

In [152]:
df = generator_sample(100)
df

,v,beta,gamma_vol,K
0,0.066876,0.840702,1.915312,0.966299
1,0.097819,0.950675,0.213806,0.991408
2,0.390145,0.193042,1.221323,0.986747
3,0.462021,0.261443,1.403983,0.939046
4,0.789591,0.985621,2.248097,1.087807
...,...,...,...,...
95,0.272991,0.156845,0.758206,0.945063
96,0.116454,0.543559,1.148611,1.022211
97,0.182302,0.243591,2.208615,1.078979
98,0.140198,0.41489,0.477811,0.987697


In [153]:
for col in df.columns: 
    df[col] = pd.to_numeric(df[col])

In [154]:
def sabr(S, K, T, alpha, beta, rho, nu):
    r = 0
    X = K
    F = S*np.exp(r*T)
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrSigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrSigma = numer/denom

    return sabrSigma



In [155]:
v_list = np.linspace(0.008, 3.032, 20)
beta_list = np.linspace(0,1, 20)
gamma_list = np.linspace(0.01, 2.5)
beta = 0.8
K = 0.966299
vv, gg = np.meshgrid(v_list, gamma_list)

In [156]:
vol_impli = sabr(S=1, K = 0.966299, T = 1, alpha = gg, beta =0.8 , rho = 0, nu= vv)

In [157]:
fig = go.Figure(data=[go.Surface(z=vol_impli, x=v_list, y=gamma_list)])

fig.update_layout(title='Implied_vol (K = 0.96, beta =0.8)', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='nu',
        yaxis_title='Vol of Vol (log)',
        zaxis_title='Implied Volatility'))

fig.show()

In [395]:
def sabr(S, K, T, alpha, beta, rho, nu):
    r = 0
    X = K
    F = S*np.exp(r*T)
    z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
    zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
    numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
    numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
    numer3 = ((2 - 3*rho*rho)/24)*nu*nu
    numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
    denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
    denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
    denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
    sabrSigma = numer/denom
    return sabrSigma


In [407]:
class Spline2d:
    def __init__(self,v, gamma):
        self.v = v
        self.gamma = gamma

    
    def computes_spline_1D(self, v_val, gamma_val):
        delta = 0.01
        Splines = {}
        vv, gg = np.meshgrid(self.v, self.gamma)
        
        r_values  = sabr(S=1, K = 0.966299, T = 1, alpha = gg, beta =0.8 , rho = 0, nu= vv)
        
        for i, u in enumerate(self.v):
            bc_1_x2 = (sabr(S=1, K = 0.966299, T = 1, alpha = self.gamma[0]+delta, beta =0.8 , rho = 0, nu= u) - sabr(S=1, K = 0.966299, T = 1, alpha = self.gamma[0]-delta, beta =0.8 , rho = 0, nu= u))/ (2*delta)
            bc_2_x2 = (sabr(S=1, K = 0.966299, T = 1, alpha = self.gamma[len(self.gamma)-1]+delta, beta =0.8 , rho = 0, nu= u) - sabr(S=1, K = 0.966299, T = 1, alpha = self.gamma[len(self.gamma)-1]-delta, beta =0.8 , rho = 0, nu= u))/ (2*delta)
            Splines[u] = CubicSpline(self.gamma, r_values[:, i], bc_type=((1, bc_1_x2), (1, bc_2_x2)))

        t = np.array([Splines[u](gamma_val)  for u in self.v])
        bc_1_x1 = (sabr(S=1, K = 0.966299, T = 1, alpha = gamma_val, beta =0.8 , rho = 0, nu= self.v[0]+delta) - sabr(S=1, K = 0.966299, T = 1, alpha = gamma_val, beta =0.8 , rho = 0, nu= self.v[0]-delta))/ (2*delta)
        bc_2_x2 = (sabr(S=1, K = 0.966299, T = 1, alpha = gamma_val, beta =0.8 , rho = 0, nu= self.v[len(self.v)-1]+delta) - sabr(S=1, K = 0.966299, T = 1, alpha = gamma_val, beta =0.8 , rho = 0, nu= self.v[len(self.v)-1]-delta))/ (2*delta)       
        Spline_f = CubicSpline(self.v, t, bc_type=((1, 0), (1, 0)))
        
        return Spline_f(v_val)
    
    def computes_spline(self, v_val, gamma_val):
        #assert len(x1_val) == len(x2_val)
        surface_spline = np.zeros((len(gamma_val), len(v_val)))
        
        for i in range(len(gamma_val)): 
            surface_spline[i, :] = self.computes_spline_1D(v_val, gamma_val[i])
    
            
        return surface_spline
        

In [408]:
#### fitting Spline
learn_mu = np.linspace(0.034, 0.677, 10)
learn_gamma = np.linspace(0.369,2.2421, 10)
beta = 0.8
K = 0.966299
learn_mu = np.insert(np.append(learn_mu, 1.432), 0, 0.016)
learn_gamma = np.insert(np.append(learn_gamma,2.448), 0, 0.09)
spline = Spline2d(learn_mu, learn_gamma)

In [409]:
test_mu = np.linspace(0.008, 3.032, 100)
test_gamma = np.linspace(0.01,2.5, 100)
vv , gg = np.meshgrid(test_mu, test_gamma)
implied_vol_exp =  sabr(S=1, K = 0.966299, T = 1, alpha = gg, beta =0.8 , rho = 0, nu= vv)

In [410]:
cs_spline_approx = spline.computes_spline(test_mu, test_gamma)

In [415]:
fig = go.Figure(data=[go.Surface(z=cs_spline_approx -implied_vol_exp, x=test_mu, y=test_gamma)])

fig.update_layout(title='Surface of the spline Prices', autosize=True,
                  width=500, height=500,
                  xaxis_title="K")
fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='Atm vol',
        yaxis_title='VOl of vol',
        zaxis_title='Residuals'))
fig.show()

In [416]:
class Spline2d:
    def __init__(self,v, gamma):
        self.v = v
        self.gamma = gamma

    
    def computes_spline_1D(self, v_val, gamma_val):
        Splines = {}
        vv, gg = np.meshgrid(self.v, self.gamma)
        
        r_values  = sabr(S=1, K = 0.966299, T = 1, alpha = gg, beta =0.8 , rho = 0, nu= vv)

        for i, u in enumerate(self.v):
            Splines[u] = CubicSpline(self.gamma, r_values[:, i], bc_type=((2, 0), (2, 0)))

        t = np.array([Splines[u](gamma_val)  for u in self.v])
        
        Spline_f = CubicSpline(self.v, t, bc_type=((2, 0), (2, 0)))
        
        return Spline_f(v_val)
    
    def computes_spline(self, v_val, gamma_val):
        #assert len(x1_val) == len(x2_val)
        surface_spline = np.zeros((len(gamma_val), len(v_val)))
        
        for i in range(len(gamma_val)): 
            surface_spline[i, :] = self.computes_spline_1D(v_val, gamma_val[i])
    
            
        return surface_spline
        

In [417]:
#### fitting Spline
learn_mu = np.linspace(0.034, 0.677, 10)
learn_gamma = np.linspace(0.369,2.2421, 10)
beta = 0.8
K = 0.966299
learn_mu = np.insert(np.append(learn_mu, 2), 0, 0.016)
learn_gamma = np.insert(np.append(learn_gamma,2.448), 0, 0.09)
spline = Spline2d(learn_mu, learn_gamma)

In [418]:
test_mu = np.linspace(0.008, 3.032, 100)
test_gamma = np.linspace(0.01,2.5, 100)
vv , gg = np.meshgrid(test_mu, test_gamma)
implied_vol_exp =  sabr(S=1, K = 0.966299, T = 1, alpha = gg, beta =0.8 , rho = 0, nu= vv)

In [419]:
cs_spline_approx = spline.computes_spline(test_mu,test_gamma)

In [421]:
fig = go.Figure(data=[go.Surface(z= implied_vol_exp-cs_spline_approx, x=test_mu, y=test_gamma)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='Atm vol',
        yaxis_title='VOl of vol',
        zaxis_title='residuals'))

fig.show()

In [337]:
class SABRDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [338]:
def simulation( T, n_simulations,K = K, range_x1=(0.034, 0.677), range_x2=(0.369,2.2421)):
    
    x1_values = np.random.uniform(range_x1[0], range_x1[1], size=n_simulations)
    x2_values = np.random.uniform(range_x2[0], range_x2[1], size=n_simulations)
    
    
    x1_column = []
    x2_column = []
    Residual = []
    
    i = 0
    combinations = product(x1_values, x2_values)
    for x1, x2 in tqdm(combinations):
        x1_column.append(x1)
        x2_column.append(x2)
        vol = sabr(S=1, K=K, T=T , alpha = x2, beta =0.8 , rho = 0, nu= x1)
        Residual.append(vol - spline.computes_spline_1D(x1, x2))
        
        i+= 1
    
    dataframe = pd.DataFrame({'x1': x1_column, 'x2': x2_column, 'Residual' : Residual})
    
    return dataframe

In [339]:
def generate_train_data(T, n_simulations, BATCH_SIZE, K=K, range_x1=(0.034, 0.677), range_x2=(0.369, 2.2421)):
    
    n_simulations = int(n_simulations * 1.2) # Additional simulations for the validation_data
    
    data = simulation(T=1, n_simulations = n_simulations, K=K,range_x1=range_x1, range_x2=range_x2)
    
    y = data.Residual
    X = data.drop(columns='Residual')
    
    # Train - val
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=69)
    
    scaler = MinMaxScaler()
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_val, y_val = np.array(X_val), np.array(y_val)
    
    y_train, y_val = y_train.astype(float), y_val.astype(float)
    
    train_dataset = SABRDataset(torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(y_train).type(torch.float32))
    val_dataset = SABRDataset(torch.from_numpy(X_val).type(torch.float32), torch.from_numpy(y_val).type(torch.float32))
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)
    
    return train_loader, val_loader, scaler


In [340]:
def generate_test_data(T, n_simulations, scaler, range_x1=(0.008, 3.032), range_x2=(0.01,2.5)):
    data = simulation(T=1, n_simulations = n_simulations, K=K,range_x1=range_x1, range_x2=range_x2)
    
    y = data.Residual
    X = data.drop(columns='Residual')
    
    #X_test = scaler.transform(X)
    X_test, y_test = np.array(X), np.array(y)
    
    y_test = y_test.astype(float)
    
    test_dataset = SABRDataset(torch.from_numpy(X_test).type(torch.float32), torch.from_numpy(y_test).type(torch.float32))
    
    test_loader = DataLoader(dataset=test_dataset, batch_size=1)
    
    return test_loader

In [341]:
T = 1

n_train_simulations = 100
n_test_simulations = 20

BATCH_SIZE = 2 ** 10



train_loader, val_loader, scaler = generate_train_data( T = T, n_simulations= n_train_simulations,BATCH_SIZE= BATCH_SIZE)
test_loader = generate_test_data( T=T, n_simulations=n_test_simulations, scaler=scaler)

print("Batch size :", BATCH_SIZE)

14400it [00:34, 415.11it/s]
400it [00:00, 606.29it/s]

Batch size : 1024


In [343]:
try:
    device = torch.device("mps")
    print('Using MPS')
except RuntimeError:
    try:
        device = torch.device("cuda")
        print("Using CUDA")
    except RuntimeError:
        device = torch.device("cpu")
        print('Using CPU')

Using MPS


In [344]:
device = torch.device("cpu")
device

device(type='cpu')

In [345]:
def tau_c(x, l, u):
    num = u * torch.exp(x) + l * torch.exp(-x)
    denom = torch.exp(x) + torch.exp(-x)
    return num/denom

In [346]:
class weightConstraint():
    def __init__(self, range_c, min_w):
        self.range_c = range_c
        self.min_w = min_w
    
    def __call__(self, module):
        if hasattr(module, 'centres'):
            c = module.centres.data
            #c = tau_c(c, self.range_c[0], self.range_c[1])
            c = c.clamp(self.range_c[0], self.range_c[1])
            module.centres.data = c
        
        if hasattr(module, 'widths'):
            
            delta = torch.min(model.CRL.centres - model.CRL.range_c[0], model.CRL.range_c[1] - model.CRL.centres)
            u_scaled = delta / model.CRL.alpha
            
            w = module.widths.data
            #w = tau_c(w, self.min_w, 1)
            w = w.clamp(self.min_w, 2)
            module.widths.data = w

In [347]:
class RBF(nn.Module):
    """
    Transforms incoming data using a given radial basis function:
    u_{i} = rbf(||x - c_{i}||^2 / s_{i}^2)
    Arguments:
        in_features: size of each input sample
        out_features: size of each output sample
    Shape:
        - Input: (N, in_features) where N is an arbitrary batch size
        - Output: (N, out_features) where N is an arbitrary batch size
    Attributes:
        centres: the learnable centres of shape (out_features, in_features).
            The values are initialised from a standard normal distribution.
            Normalising inputs to have mean 0 and standard deviation 1 is
            recommended.
        
        log_sigmas: logarithm of the learnable scaling factors of shape (out_features).
        
        basis_func: the radial basis function used to transform the scaled
            distances.
    """

    def __init__(self, in_features, out_features, basis_func, range_c=(-4, 4), min_w=0.1, alpha=3):
        super(RBF, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.centres = nn.Parameter(torch.Tensor(out_features, in_features))
        self.widths = nn.Parameter(torch.Tensor(out_features, in_features))
        
        self.basis_func = basis_func
        self.range_c = range_c
        self.min_w = min_w
        self.alpha = alpha
        
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.uniform_(self.centres, self.range_c[0], self.range_c[1])
        nn.init.uniform_(self.widths, self.min_w, 10 * self.min_w)

    def forward(self, input):
        size = (input.size(0), self.out_features, self.in_features)
        x = input.unsqueeze(1).expand(size)
        
        constraints = weightConstraint(self.range_c, self.min_w)
        self = self.apply(constraints)
        
        c = self.centres.unsqueeze(0).expand(size)
        
        distances = (x - c).pow(2) / self.widths.pow(2).unsqueeze(0)
        distances = distances.sum(-1)
        return self.basis_func(-distances)

In [348]:
class SABRRegression(nn.Module):
    def __init__(self, num_features, range_c):
        super(SABRRegression, self).__init__()
        n_nodes = 64
        self.CRL = RBF(num_features, n_nodes, basis_func=torch.exp, range_c=range_c)
        self.layer_out = nn.Linear(n_nodes, 1, bias=False)
        #self.activation = nn.LeakyReLU()
        
    def forward(self, inputs):
        x = self.CRL(inputs)
        #x = self.activation(x)
        x = self.layer_out(x)
        return x

    def predict(self, test_inputs):
        x = self.CRL(test_inputs)
        #x = self.activation(x)
        x = self.layer_out(x)
        return x

In [349]:
NUM_FEATURES = 2 

In [350]:
model = SABRRegression(NUM_FEATURES, (-0.5,0.5))
model.to(device)

SABRRegression(
  (CRL): RBF()
  (layer_out): Linear(in_features=64, out_features=1, bias=False)
)

In [351]:
loss_stats = {
    'train': [],
    "val": []
}

In [352]:
EPOCHS = 500
LEARNING_RATE = 1e-3

In [353]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [354]:
print("Begin training.")

for e in range(1, EPOCHS+1):
    
    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))
            
            val_epoch_loss += val_loss.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

Begin training.
Epoch 001: | Train Loss: 0.00578 | Val Loss: 0.00320
Epoch 002: | Train Loss: 0.00233 | Val Loss: 0.00136
Epoch 003: | Train Loss: 0.00096 | Val Loss: 0.00072
Epoch 004: | Train Loss: 0.00052 | Val Loss: 0.00042
Epoch 005: | Train Loss: 0.00037 | Val Loss: 0.00029
Epoch 006: | Train Loss: 0.00026 | Val Loss: 0.00021
Epoch 007: | Train Loss: 0.00018 | Val Loss: 0.00015
Epoch 008: | Train Loss: 0.00013 | Val Loss: 0.00011
Epoch 009: | Train Loss: 0.00009 | Val Loss: 0.00008
Epoch 010: | Train Loss: 0.00007 | Val Loss: 0.00006
Epoch 011: | Train Loss: 0.00006 | Val Loss: 0.00005
Epoch 012: | Train Loss: 0.00004 | Val Loss: 0.00004
Epoch 013: | Train Loss: 0.00004 | Val Loss: 0.00004
Epoch 014: | Train Loss: 0.00003 | Val Loss: 0.00003
Epoch 015: | Train Loss: 0.00003 | Val Loss: 0.00003
Epoch 016: | Train Loss: 0.00003 | Val Loss: 0.00002
Epoch 017: | Train Loss: 0.00002 | Val Loss: 0.00002
Epoch 018: | Train Loss: 0.00002 | Val Loss: 0.00002
Epoch 019: | Train Loss: 0.000

KeyboardInterrupt: 

In [ ]:
plt.plot(loss_stats['train'])

In [372]:

T = 1

n_x1_points = 100
n_x2_points = 100

x1_values = np.linspace(0.008, 3.032, n_x1_points)
x2_values = np.linspace(0.01,2.5, n_x2_points)

X_test = np.array(list(product(x1_values, x2_values))).astype('float32')

y_test_pred = model.predict(torch.from_numpy(X_test).to(device)).cpu().detach().numpy()

y_test_pred = y_test_pred.reshape((n_x1_points, n_x2_points)).T


In [414]:
fig = go.Figure(data=[go.Surface(z=y_test_pred, x=x1_values, y=x2_values)])

fig.update_layout(title='Surface', autosize=True,
                  width=700, height=700,
                  scene=dict(
        xaxis_title='Atm vol',
        yaxis_title='VOl of vol',
        zaxis_title='Residuals'))

fig.show()